In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Managers.GameDirector import GameDirector
from Bots import RandomBot, AlexPastorBot, EdoBot, AdrianHerasBot
import pandas as pd
from tqdm import tqdm
import random

# bots_configs = [
#     [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot],
#     ]

bots_configs = [
    [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
    [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
    [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
    ]

def shuffle_bots(bots):
    names = [f"{bot.__name__}_{j}" for j, bot in enumerate(bots)]
    temp = list(zip(names, bots))
    random.shuffle(temp)
    res1, res2 = zip(*temp)
    names, bots = list(res1), list(res2)
    return names, bots

def play_single_game(bots):
    names, bots = shuffle_bots(bots)
    game_director = GameDirector(bots = bots)
    game_director.game_start(0)
    last_round = list(game_director.trace_loader.current_trace["game"].values())[-1]
    last_turn = list(last_round.values())[-1]
    points = list(last_turn["end_turn"]["victory_points"].values())
    points = [int(point) for point in points]
    return {name: points[i] for i, name in enumerate(names)}


def play_set_of_games(bots, games_to_play=100):
    return [play_single_game(bots) for i in tqdm(range(games_to_play))]

In [3]:
from joblib import delayed, Parallel

# results_list = Parallel(n_jobs=3)(delayed(play_set_of_games)(bots) for bots in bots_configs)
results_list = [ play_set_of_games(bots) for bots in bots_configs]

for results in results_list:
    df = pd.DataFrame(results)
    max_counts = dict(df.idxmax(axis=1).value_counts())
    bot_wins = {bot: max_counts.get(bot, 0)/len(df.index)*100 for bot in list(df.columns)}

    display(pd.DataFrame(bot_wins, index=["% wins"]).T)

 25%|██▌       | 25/100 [00:04<00:08,  8.82it/s]

KeyboardInterrupt: 

In [4]:
from Bots.helpers import *
from Bots.logger import *

clear_log_data()
points = play_single_game([EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot])
winer = max(points, key=points.get)
print(points)
print(winer)

{'RandomBot_3': 10, 'RandomBot_1': 4, 'RandomBot_2': 8, 'EdoBot_0': 3}
RandomBot_3


In [6]:
log_data = get_log_df()
display(log_data)

,id,🥖,🪨,🧱,🪵,🧶,g1,g2,g3,events
0,1,1,3,0,4,2,🏢 (2🥖3🪨),🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),[Building town]
1,2,1,0,0,6,2,🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),🏢 (2🥖3🪨),[Add build_road to goals]
2,3,1,0,0,6,2,🛣️ (1🧱1🪵),🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),[]
3,4,1,0,0,8,2,🛣️ (1🧱1🪵),🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),[Thief called]
4,5,0,0,0,0,2,🛣️ (1🧱1🪵),🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),[]
...,...,...,...,...,...,...,...,...,...,...
201,202,0,0,0,0,4,🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),🏢 (2🥖3🪨),[Moving thief]
202,203,0,0,0,10,4,🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),🏢 (2🥖3🪨),[]
203,204,0,0,0,18,4,🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),🏢 (2🥖3🪨),[]
204,205,0,0,0,22,16,🛖 (1🥖1🧱1🪵1🧶),🛖 (1🥖1🧱1🪵1🧶),🏢 (2🥖3🪨),[]


In [10]:
log_data[["🥖", "🪨", "🧱", "🪵", "🧶"]].max()

🥖     8
🪨    11
🧱     1
🪵    35
🧶    43
dtype: int64